<center>
<img src="https://raw.githubusercontent.com/dvgodoy/PyTorch101_ODSC_Europe2020/master/images/linear_dogs.jpg" width="800">

# Основы глубинного обучения: домашнее задание 4
</center>

**ФИО:** Малиницкий Дмитрий Андреевич

**Забавный факт о себе:** у меня 3 ноутбука и 2 3д принтера

## Общая информация

__Дата выдачи:__ 26.11.2024

__Мягкий дедлайн:__ 23:59MSK 10.12.2024

__Жесткий дедлайн:__ 23:59MSK 13.12.2024

## Оценивание и штрафы

Каждая из задач имеет определенную «стоимость» (указана в скобках около задачи). Максимально допустимая оценка за работу — 10 баллов + 2 бонусных балла.

**Дисклеймер:** Задание выполняется самостоятельно. «Похожие» решения считаются плагиатом и все задействованные студенты (в том числе те, у кого списали) не могут получить за него больше 0 баллов. Если вы нашли решение какого-то из заданий (или его часть) в открытом источнике, необходимо указать ссылку на этот источник (скорее всего вы будете не единственным, кто это нашел, поэтому чтобы исключить подозрение в плагиате, необходима ссылка на источник). Также помните, что простое копирование чужого кода не является самостоятельной работой.

In [110]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from tqdm.notebook import tqdm

In [111]:
import torch
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda', index=0)

# Описание данных

Данные можно [скачать с гугл-диска.](https://drive.google.com/drive/folders/11oCcLplWtp_qm-WuEbfCFP_Mz5K_z3ps?usp=sharing) Если вы делаете задание в колабе, то строчки ниже сами скачают вам данные.



В таблице `ria_news.tsv`  лежат данные о новостях, вышедших на сайте РИА-НОВОСТИ с 15 марта 2018 года по 31 декабря 2018 года.

- `href` - уникальный идентификатор новости (ссылка на неё)
- `date` - дата публикации новости
- `time` - время публикации новости
- `title` - заголовок новости
- `snippet` - краткое описание новости
- `text` - текст новости
- `category` - категория новости
- `keywords` - ключевые слова (подкатегории новости)
- `shows` - счётчик с числом просмотров новости на сайте (на момент парсинга)

Эххх, жалко что датасет старый, вот бы с 20 года, или с 22, эх мечты мечты...

In [112]:
df_ria = pd.read_csv('news_data/ria_news.tsv', sep='\t')
df_ria = df_ria[~df_ria.tags.isnull()]
print(df_ria.shape)
df_ria.head()

(201708, 9)


,href,date,time,title,snippet,text,category,tags,shows
0,/20181231/1548961410.html,2018-12-31,"31 декабря 2018, 23:52",Нетаньяху не собирается в отставку в случае пр...,Премьер-министр Израиля Биньямин Нетаньяху не ...,"МОСКВА, 31 дек - РИА Новости. Премьер-министр ...",В мире,"Биньямин Нетаньяху, Израиль, В мире",728.0
1,/20181231/1548961364.html,2018-12-31,"31 декабря 2018, 23:19",Макрон в новогоднем обращении затронул тему ре...,"Результат реформ не может быть мгновенным, зая...","ПАРИЖ, 31 дек – РИА Новости. Результат реформ ...",В мире,"Эммануэль Макрон, Франция, В мире",3086.0
2,/20181231/1548961337.html,2018-12-31,"31 декабря 2018, 23:12",Аарон Рэмзи проведет переговоры с пятью топ-кл...,"Полузащитник лондонского ""Арсенала"" Аарон Рэмз...","МОСКВА, 31 дек - РИА Новости. Полузащитник лон...",NaN,ФК Арсенал (Лондон),183.0
3,/20181231/1548961304.html,2018-12-31,"31 декабря 2018, 23:09",Гол Азмуна принес сборной Ирана победу над кат...,Футболисты сборной Ирана одержали победу над к...,"МОСКВА, 31 дек - РИА Новости. Футболисты сборн...",NaN,"Сердар Азмун, Сборная Ирана по футболу",78.0
4,/20181231/1548961265.html,2018-12-31,"31 декабря 2018, 23:07",Пятая ракетка мира дель Потро пропустит Открыт...,Аргентинский теннисист Хуан Мартин дель Потро ...,"МОСКВА, 31 дек - РИА Новости. Аргентинский тен...",NaN,Теннис,79.0


Многие новостные агенства поддерживают странички в социальных сетях. Они постят туда самые сочные сюжеты. В таблице `vk_news.tsv` лежат данные о новостях, которые РИА запостили ВКонтакте в период времени с  `2017-09-29 01:28:55` по `2019-02-01 23:13:17`.

- `id` - уникальный идентификатор поста
- `href` - ссылка на сайт (если она была указана в посте)
- `datetime` - дата и время публикации новости
- `title` - заголовок новости
- `text` - текст новости в социальной сети
- `likes` - число лайков под постом
- `comments` - число комментариев под постом

In [113]:
df_vk = pd.read_csv('news_data/vk_news.tsv', sep='\t')
df_vk['snippet'] = df_vk['text']
df_vk.drop('text', axis=1, inplace=True)
print(df_vk.shape)
df_vk.head()

(19928, 7)


,id,href,datetime,title,likes,comments,snippet
0,24006362,/20190201/1550280358.html,2019-02-01 23:13:17,"В ДНР заявили о задержании диверсантов, причас...",15,28,NaN
1,24006240,/20190201/1550268781.html,2019-02-01 22:38:41,"Житель Урала ""заминировал"" ТЦ из-за снятия со...",32,42,NaN
2,24006100,/20190201/1550282212.html,2019-02-01 21:58:52,"В Черном море нашли ""потерянный флот Гитлера""",84,23,NaN
3,24005972,/20190202/1550283179.html,2019-02-01 21:27:06,В США освободили задержанную российскую актрис...,58,35,NaN
4,24005764,/20190201/1550262848.html,2019-02-01 20:55:54,Толкнувший Скабееву депутат Рады заявил о гроз...,45,145,NaN


В таблице `vk_comments.tsv` лежат комментарии к новостям.

- `id` - уникальный идентификатор комментария
- `post_id` - идентификатор новости, под которой был оставлен комментарий
- `datetime` - дата и время, когда был оставлен комментарий
- `text` - текст комментария
- `likes` - число лайков под комментарием

In [114]:
df_comments = pd.read_csv('news_data/vk_comments.tsv', sep='\t')
df_comments = df_comments[~df_comments.text.isnull()]
print(df_comments.shape)
df_comments.head()

/tmp/ipykernel_15604/851848438.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df_comments = pd.read_csv('news_data/vk_comments.tsv', sep='\t')


(2612629, 5)


,id,post_id,datetime,text,likes
0,24006366.0,24006362.0,2019-02-01 23:14:14,ЧВК Вагнера?,5.0
1,24006370.0,24006362.0,2019-02-01 23:15:23,"[id4710641|Евгений], выздоравливай.",3.0
2,24006371.0,24006362.0,2019-02-01 23:16:21,"[id442655034|Андрей], искренне желаю этого все...",4.0
3,24006374.0,24006362.0,2019-02-01 23:16:38,Опять про Украину новости?,1.0
4,24006375.0,24006362.0,2019-02-01 23:16:40,Че такое ДНР?,2.0


# А что надо сделать то?

В тетрадке вам предстоит сделать следующие шаги:

1. Обучить нейросеть предсказывать категорию новости
2. Построить предсказания для тех новостей, где мы ничего не знаем о категории
3. Использовать уже обученный для сентимент-анализа классификатор из библиотеки `hugging face` чтобы предсказать эмоциональную окраску каждого комментария
4. Провести аналитику по новостям, а именно построите топы из самых позитивных и негативных категорий и новостей

Для первого шага вам будет дан бэйзлайн. Если вы его прогоните, у вас получится базовая модель, которая даст некоторое качество решения задачи. Вам надо будет выяснить, насколько это качество оказалось хорошим, а затем внести в код некоторые улучшения.



## Часть 1: категоризация новостей (5 баллов + 1.8 бонусных)

Каждой новости в соотвествие поставлены ключевые слова. Будем считать, что эти ключевые слова — тематики новости. Нужно научиться предсказывать тематики по тексту новости. Готовые тематики у нас есть только по новостям с сайта. Они за 2018 год. По новостям из ВКонтакте у нас тематик нет. Мы собираемся их предсказать.

Новости, опубликованные ВКонтакте, отличаются от новостей с сайта тем, что у них есть только титул и короткое описание. Странно будет обучать нейросеть на длинных текстах, а потом использовать её на коротких описаниях. Мы не будем так делать. Мы попробуем обучить базовый вариант нейронной сети только на заголовках новостей. Все, кто захочет получить бонусные баллы, смогут попробовать добавить в нейросеть сниппеты (так назыают короткие описания новостей).

## 1.1 Подготовка таргета

Поработаем с таргетом. Мы будем предсказывать переменную `tags`. Давайте выясним скоько уникальных тегов существует.

In [115]:
from collections import Counter

# удалим все лишние пробелы и сделаем lowercase
df_ria['tags'] = (
    df_ria.tags.
    apply(lambda w: ','.join([item.strip() for item in  w.lower().split(',')]))
)

tags = ','.join(list(df_ria.tags.values))
tags_cnt = Counter(tags.split(','))

print(len(tags_cnt))
tags_cnt.most_common()[-20:]

13344


[('блог анны завершинской об автоспорте - блоги', 1),
 ('министерство транспорта рб', 1),
 ('министерство здравоохранения грузии', 1),
 ('палех', 1),
 ('юрий посохов (хореограф)', 1),
 ('мария александрова', 1),
 ('том бенсон', 1),
 ('абдул каюм кочай', 1),
 ('нуман куртулмуш', 1),
 ('mipim', 1),
 ('владимир попов', 1),
 ('брюно женезио', 1),
 ('роберт фицо', 1),
 ('сергей пашинский', 1),
 ('валерия гонтарева', 1),
 ('нововоронеж', 1),
 ('императорское православное палестинское общество', 1),
 ('event_poslanie_prezidenta_rf_federalnomu_sobraniju', 1),
 ('фхтр', 1),
 ('игорь честин', 1)]

Всего в выборке есть порядка 13 000 тэгов. Многие встречаются всего по разу. Давайте оставим в выборке только те тэги, которые встречаются более 30 раз.

In [116]:
target_tags = {tag for tag,cnt in tags_cnt.most_common() if cnt > 30}
len(target_tags)

1583

Закодируем теги для OHE.

In [117]:
tag2idx = dict(zip(target_tags, range(len(target_tags))))
idx2tag = {jtem: item for item,jtem in tag2idx.items()}

Почистим таргет от лишних тэгов.

In [118]:
df_ria['target_tags'] = (
    df_ria.tags.
    apply(lambda w: [tag2idx.get(item) for item in  w.split(',') if item in target_tags])
)

df_ria = df_ria[df_ria.target_tags.apply(len) > 0]
df_ria.shape

(201437, 10)

In [119]:
df_ria.target_tags.values[:3]

array([list([438, 1298, 942]), list([984, 1146, 942]), list([1390])],
      dtype=object)

## 1.2 Подготовка текстов

Теперь займёмся предобработкой текстов. Приведём все слова к нижнему регистру и выбросим мусорные символы. В качестве токенов будем рассматривать отдельные слова.

Напомню, что мы пока что решили работать только с названиями статей. Поэтому вся предобработка применяется исключительно к ним. **Спойлер:** предобработку для сниппетов вы сделаете сами в первом же задании.

In [120]:
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /home/user/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /home/user/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /home/user/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [121]:
import re
from nltk.tokenize import word_tokenize

def normalise_text(text):
    text = text.lower()

    # сурово регулярками выкидываем мусорные символы
    text = re.sub('[^а-яa-z0-9 ]', '', text)
    return text.strip()

df_ria['title_clean'] = df_ria.title.apply(normalise_text)

word_cnt = Counter(word_tokenize(' '.join(df_ria.title_clean.values)))
len(word_cnt)

112178

In [122]:
word_cnt.most_common()[:10]

[('в', 127323),
 ('на', 44386),
 ('с', 26150),
 ('и', 21771),
 ('о', 19948),
 ('по', 17014),
 ('россии', 13494),
 ('не', 13483),
 ('сша', 9942),
 ('за', 9881)]

# Ураааа, РОССИЯ

In [123]:
import webbrowser 
webbrowser.open("https://www.youtube.com/watch?v=xSrsouS0_4U")


True

### Ладно, хватит патриотизма

Давайте почистим словарь от стоп-слов и подготовим его к использованию внутри датасета. Мы будем с помощью словаря заменять слова на индексы. Добавим в словарь несколько специальных токенов для неизвестных слов и паддингов.

In [124]:
from nltk.corpus import stopwords

stops_ru = set(stopwords.words('russian'))
len(stops_ru)

151

In [125]:
vocabulary = {
    "#PAD#": 0, "#UNK#": 1
}

k = 2
for word, _ in word_cnt.most_common():
    if word not in stops_ru:
        vocabulary[word] = k
        k += 1

In [126]:
len(vocabulary)

112030

Завернём код для создания словаря в функцию.

In [127]:
def create_vocab(text, stops_ru=stops_ru):

    word_cnt = Counter(word_tokenize(text))
    vocabulary = {
        "#PAD#": 0, "#UNK#": 1
    }

    k = 2
    for word, _ in word_cnt.most_common():
        if word not in stops_ru:
            vocabulary[word] = k
            k += 1
    return vocabulary

__[0.5 балла] Задание 1:__

- Cделайте аналогичную предобработку титулов из таблички `df_vk`. Запишите получившийся результат в столбец `title_clean` по аналогии с таблицей `df_ria`.
- Сделайте для обеих таблиц предобработку колонок со сниппетами `snippet` и запишите получившийся результат в столбец `snippet_clean`. Все пропуски заполните токеном `"#UNKN"`.

In [128]:
df_vk['title_clean'] = df_vk.title.apply(normalise_text)

df_ria['snippet_clean'] = df_ria.snippet.fillna("#UNKN").apply(normalise_text)
df_vk['snippet_clean'] = df_vk.snippet.fillna("#UNKN").apply(normalise_text)

In [129]:
print(df_vk[['title', 'title_clean']].head())

                                               title  \
0  В ДНР заявили о задержании диверсантов, причас...   
1  Житель Урала "заминировал" ТЦ из-за снятия  со...   
2      В Черном море нашли "потерянный флот Гитлера"   
3  В США освободили задержанную российскую актрис...   
4  Толкнувший Скабееву депутат Рады заявил о гроз...   

                                         title_clean  
0  в днр заявили о задержании диверсантов причаст...  
1  житель урала заминировал тц изза снятия  сотов...  
2        в черном море нашли потерянный флот гитлера  
3  в сша освободили задержанную российскую актрис...  
4  толкнувший скабееву депутат рады заявил о гроз...  


In [130]:
print(df_ria[['snippet', 'snippet_clean']].head())


                                             snippet  \
0  Премьер-министр Израиля Биньямин Нетаньяху не ...   
1  Результат реформ не может быть мгновенным, зая...   
4  Аргентинский теннисист Хуан Мартин дель Потро ...   
5  Суд в американском штате Массачусетс отверг хо...   
6  "Зимняя классика" - матч регулярного чемпионат...   

                                       snippet_clean  
0  премьерминистр израиля биньямин нетаньяху не н...  
1  результат реформ не может быть мгновенным заяв...  
4  аргентинский теннисист хуан мартин дель потро ...  
5  суд в американском штате массачусетс отверг хо...  
6  зимняя классика  матч регулярного чемпионата н...  


In [131]:
print(df_vk[['snippet', 'snippet_clean']].head(20))

                                              snippet  \
0                                                 NaN   
1                                                 NaN   
2                                                 NaN   
3                                                 NaN   
4                                                 NaN   
5                                                 NaN   
6                                                 NaN   
7                                                 NaN   
8                                                 NaN   
9                                                 NaN   
10                                                NaN   
11                                                NaN   
12                                                NaN   
13                                                NaN   
14                                                NaN   
15                                                NaN   
16  Ранее в соцсетях появились 

Ладно всё окей, а тоя испугался


## 1.3 Поставка данных

Пересечение сайта и ВК по опубликованным новостям довольно маленькое. Мы обучаем модель на данных с сайта. Предсказания мы будем строить на данных из ВК. У этих данных разная природа. В ВК описание статей и заголовки короче. Модель может хорошо показать себя на данных с новостного сайта, но сильно просесть в качестве на данных из ВК.

Давайте сохраним пересечение в отдельную табличку, чтобы на нём можно было понять, насколько сильно деградирует модель.

In [132]:
ria_hrefs = set(df_ria.href.values)
vk_hrefs = set(df_vk.href.values)
test_hrefs = (vk_hrefs & ria_hrefs)

print('Размер отложенной выборки:', len(test_hrefs))

df = df_ria[~df_ria.href.isin(test_hrefs)]

Размер отложенной выборки: 1128


По странному совпадению (я правда не знаю почему) пересечение лежит в декабре. Мы будем его использовать как тестовую выборку.

In [133]:
df_ria[df_ria.href.isin(test_hrefs)].date.min(), df_ria[df_ria.href.isin(test_hrefs)].date.max()

('2018-12-06', '2018-12-31')

Предположим, что мы делим выборку на обучающую и тестовую случайно. За один и тот же промежуток времени может выйти довольно большое число новостей с одинаковым заголовком. Давайте представим себе, что в тесте и трэйне есть много статей про одно и то же событие. Модель научилась на обучающей выборке хорошо его тегировать. Остальные события модель тегирует намного хуже. Метрики на тестовой выборке высокие. В следующем месяце СМИ перестают освещать это событие, в потоке новостей совершенно другие новости. Качество модели резко проседает.

Чтобы не напороться на завышенные метрики, обычно выборку дробят на обучающую и тестовую по времени. Тогда статьи из теста будут имитировать поток новых новостей, освещающих новые события.

In [134]:
df.date.min(), df.date.max()

('2018-03-15', '2018-12-31')

__[0.5 балла] Задание 2:__ Разбейте выборку на обучающую, валидационную и тестовую. В тест возьмите весь декабрь. В валидацию октябрь и ноябрь.

In [135]:
df['date'] = pd.to_datetime(df['date'])
df_test = df[df['date'].dt.month == 12]
df_val = df[df['date'].dt.month.isin([10, 11])]
df_train = df[~df['date'].dt.month.isin([10, 11, 12])]

/tmp/ipykernel_15604/3942277652.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['date'] = pd.to_datetime(df['date'])


In [136]:
print(f"Train set: {df_train.shape}")
print(f"Validation set: {df_val.shape}")
print(f"Test set: {df_test.shape}")

Train set: (136875, 12)
Validation set: (43258, 12)
Test set: (20176, 12)


Сформируем отложенную выборку (пересечение ВКонтакте и РИА).

In [137]:
df_oob = df_vk[df_vk.href.isin(test_hrefs)][['href', 'title_clean']]

df_ria_oob = df_ria[df_ria.href.isin(test_hrefs)][['href', 'target_tags']]
df_oob = df_oob.set_index('href').join(df_ria_oob.set_index('href')).reset_index()
df_oob.head()

,href,title_clean,target_tags
0,/20181231/1548960744.html,митрополит рязанский раскритиковал иронию судьбы,[1115]
1,/20181231/1548958617.html,на украине позавидовали стене на границе с крымом,"[1359, 942]"
2,/20181231/1548957394.html,в госдуме предложили отказаться от газа в жилы...,"[1556, 1556, 1201, 194, 973]"
3,/20181231/1548954909.html,названы лучшие средства от похмелья,[1115]
4,/20181231/1548957120.html,новогоднее поздравление порошенко разозлило по...,"[1359, 942]"


Напишем датасет для поставки данных в нейросеть.


In [138]:
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

class NewsDataset(Dataset):

    def __init__(self, target, title, vocab, vocab_size, max_title_len, max_classes, snippet=None, max_snippet_len=None):

       self.vocab = {word: idx  for word,idx in vocab.items() if idx < vocab_size}
       self.max_classes = max_classes
       self.y=self.target_ohe(target)
       self.X_title = self.create_text(title, max_title_len)

    def target_ohe(self, target):
        y = torch.zeros((len(target), self.max_classes))
        for i, t in enumerate(target):
            y[[i]*len(t), t] = 1.0
        return y

    def create_text(self, texts, max_len):
        result = [ ]
        for sent in texts:
            # {#PAD: 0, #UNKN: 1}
            sent_tokenize = [self.vocab.get(item, 1) for item in word_tokenize(sent)]

            # приводим все тексты к max_len
            if len(sent_tokenize) >= max_len:
                sent_tokenize = sent_tokenize[:max_len]
            else:
                sent_tokenize += [0] * (max_len - len(sent_tokenize))
            result.append(sent_tokenize)
        return torch.tensor(result, dtype=torch.int)

    def __len__(self):
        return len(self.X_title)

    def __getitem__(self, idx):
        return (self.X_title[idx, :], self.y[idx])


__[0.5 балла] Задание 3:__ Сейчас датасет умеет работать только с полем `title_clean`. Давайте сделаем этот датасет более многофукнциональным и добавим в него возможность добавить в обработку данных сниппет.

1. Внутри датасета `snippet` надо обработать точно также как и `title`.
2. Если `snippet=None`, датасет должен вернуть два объекта: `X_title, y`. В обратном случае датасет должен вернуть три объекта.

**Важно:** Весь код ниже работает сейчас без сниппета. Он не должен развалиться от того, что сниппет в нём нигде не указан.

In [139]:
class NewsDataset(Dataset):

    def __init__(self, target, title, vocab, vocab_size, max_title_len, max_classes, 
                 snippet=None, max_snippet_len=None):
        self.vocab = {word: idx for word, idx in vocab.items() if idx < vocab_size}
        self.max_classes = max_classes
        self.y = self.target_ohe(target)
        self.X_title = self.create_text(title, max_title_len)
        
        if snippet is not None and max_snippet_len is not None:
            self.has_snippet = True
            self.X_snippet = self.create_text(snippet, max_snippet_len)
        else:
            self.has_snippet = False

    def target_ohe(self, target):
        y = torch.zeros((len(target), self.max_classes))
        for i, t in enumerate(target):
            y[[i]*len(t), t] = 1.0
        return y

    def create_text(self, texts, max_len):
        result = []
        for sent in texts:
            sent_tokenize = [self.vocab.get(item, 1) for item in word_tokenize(sent)]
            if len(sent_tokenize) >= max_len:
                sent_tokenize = sent_tokenize[:max_len]
            else:
                sent_tokenize += [0] * (max_len - len(sent_tokenize))
            result.append(sent_tokenize)
        return torch.tensor(result, dtype=torch.int)

    def __len__(self):
        return len(self.X_title)

    def __getitem__(self, idx):
        if self.has_snippet:
            return (self.X_title[idx, :], self.X_snippet[idx, :], self.y[idx])
        else:
            return (self.X_title[idx, :], self.y[idx])

Объявим датасеты, оставим в словаре 30 000 самых частотных слов. Будем смотреть на титулы максимальной длины 20.

In [140]:
CLASSES_NUM = len(idx2tag)
VOCAB_SIZE = 10000
MAX_TITLE_LEN = 20

# словарь создаем по всей выборке
vocabulary = create_vocab(' '.join(df_ria.title_clean.values))

# объявляем датасеты
train_dataset = NewsDataset(df_train.target_tags.values, df_train.title_clean.values, vocabulary, VOCAB_SIZE, MAX_TITLE_LEN, CLASSES_NUM )
val_dataset = NewsDataset(df_val.target_tags.values, df_val.title_clean.values, vocabulary, VOCAB_SIZE, MAX_TITLE_LEN, CLASSES_NUM )
test_dataset = NewsDataset(df_test.target_tags.values, df_test.title_clean.values, vocabulary, VOCAB_SIZE, MAX_TITLE_LEN, CLASSES_NUM )

In [141]:
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=64, num_workers=4)
val_dataloader = DataLoader(val_dataset, shuffle=False, batch_size=4096, num_workers=4)

## 1.4 Архитектуры

Соберём базовую архитектуру для обучения.

In [142]:
from torch import nn
import torch.nn.functional as F

class SimpleClassifier(nn.Module):

    def __init__(self, vocab_size, embedding_dim, output_dim):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.fc = nn.Linear(embedding_dim, output_dim)

    def forward(self, title):
        embedded = self.embedding(title)
        embedded = embedded.mean(dim=1)
        return self.fc(embedded)

Соберём в `pytorch_lightning` модуль для обучения нейронки.

In [143]:
import pytorch_lightning as pl

class TrainLightningModule(pl.LightningModule):
    def __init__(self, model, learning_rate, criterion):
        super().__init__()
        self.model = model
        self.criterion = criterion
        self.learning_rate = learning_rate

    def forward(self, title):
        result = self.model(title)
        return result

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.learning_rate)
        return optimizer

    def training_step(self, train_batch, batch_idx):
        title, target = train_batch
        logits = self.model(title)
        loss = self.criterion(logits, target)
        self.log(
            "train_loss", loss, prog_bar=True
        )
        return loss

    def validation_step(self, val_batch, batch_idx):
        title, target = val_batch
        logits = self.model(title)
        loss = self.criterion(logits, target)
        self.log(
            "val_loss", loss, prog_bar=True
        )
        return loss

Обучим модель.

In [144]:
trainer = pl.Trainer(accelerator="gpu", devices=1)


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [145]:
import torch
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

x = torch.rand(3, 3).to(device)
print(x)


Using device: cuda:0
tensor([[0.2666, 0.6274, 0.2696],
        [0.4414, 0.2969, 0.8317],
        [0.1053, 0.2695, 0.3588]], device='cuda:0')


In [146]:
EMBEDDING_DIM = 300
EPOCHS = 5
LR = 1e-3

model_baseline = SimpleClassifier(VOCAB_SIZE, EMBEDDING_DIM, CLASSES_NUM)
criterion = torch.nn.CrossEntropyLoss()
train_module = TrainLightningModule(model_baseline, LR, criterion)

trainer = pl.Trainer(accelerator="gpu", max_epochs=EPOCHS)
trainer.fit(train_module, train_dataloader, val_dataloader)


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type             | Params | Mode 
-------------------------------------------------------
0 | model     | SimpleClassifier | 3.5 M  | train
1 | criterion | CrossEntropyLoss | 0      | train
-------------------------------------------------------
3.5 M     Trainable params
0         Non-trainable params
3.5 M     Total params
13.906    Total estimated model params size (MB)
4         Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

Epoch 0: 100%|██████████| 2139/2139 [00:14<00:00, 147.58it/s, v_num=18, train_loss=12.20]

...

Epoch 4: 100%|██████████| 2139/2139 [00:15<00:00, 137.37it/s, v_num=18, train_loss=7.090, val_loss=10.90]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

Epoch 4: 100%|██████████| 2139/2139 [00:17<00:00, 120.34it/s, v_num=18, train_loss=7.090, val_loss=10.70]

`Trainer.fit` stopped: `max_epochs=5` reached.


Epoch 4: 100%|██████████| 2139/2139 [00:17<00:00, 119.36it/s, v_num=18, train_loss=7.090, val_loss=10.70]


__[1 балл] Задание 4:__  Модель выше работает только с титулом.

- Залоггируйте её обучение на [WandB](https://wandb.ai/).
- Соберите архитектуру, которая будет принимать на вход не только титул, но ещё и сниппет. В этой архитектуре должно происходить следующее:

1. Общий слой `nn.Embedding` применяется к сниппету и титулу параллельно.
2. Происходит усреднее по текстам.
3. Вектора конкатятся в один длины 600
4. Линейный слой делает классификацию

Обучите эту модель. Сравните траектории обучения на WandB. Прикрепите ссылку на дашборд либо скришот к тетрадке.

Даталоадеры придётся объявить заново с учётом сниппетов. Правда ли, что она бьёт на валидационной выборке модель, обученную только на титулах статей?

In [147]:
import wandb
from wandb_key import API_KEY 
from pytorch_lightning.loggers import WandbLogger

wandb.login(key=API_KEY)
wandb_logger = WandbLogger(project="news-categorization", log_model="all")


wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


In [148]:
MAX_SNIPPET_LEN = 30
train_dataset_snippet = NewsDataset(
    target=df_train.target_tags.values, 
    title=df_train.title_clean.values, 
    vocab=vocabulary, 
    vocab_size=VOCAB_SIZE, 
    max_title_len=MAX_TITLE_LEN, 
    max_classes=CLASSES_NUM,
    snippet=df_train.snippet_clean.values,
    max_snippet_len=MAX_SNIPPET_LEN
)

val_dataset_snippet = NewsDataset(
    target=df_val.target_tags.values, 
    title=df_val.title_clean.values, 
    vocab=vocabulary, 
    vocab_size=VOCAB_SIZE, 
    max_title_len=MAX_TITLE_LEN, 
    max_classes=CLASSES_NUM,
    snippet=df_val.snippet_clean.values,
    max_snippet_len=MAX_SNIPPET_LEN
)

test_dataset_snippet = NewsDataset(
    target=df_test.target_tags.values, 
    title=df_test.title_clean.values, 
    vocab=vocabulary, 
    vocab_size=VOCAB_SIZE, 
    max_title_len=MAX_TITLE_LEN, 
    max_classes=CLASSES_NUM,
    snippet=df_test.snippet_clean.values,
    max_snippet_len=MAX_SNIPPET_LEN
)

train_dataloader_snippet = DataLoader(train_dataset_snippet, shuffle=True, batch_size=64, num_workers=4)
val_dataloader_snippet = DataLoader(val_dataset_snippet, shuffle=False, batch_size=4096, num_workers=4)


In [149]:

class SnippetClassifier(nn.Module):
    def __init__(self, vocab_size, embedding_dim, output_dim):
        super(SnippetClassifier, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.fc = nn.Linear(embedding_dim * 2, output_dim)
        
    def forward(self, title, snippet):
        embedded_title = self.embedding(title)
        embedded_title = embedded_title.mean(dim=1)
        
        embedded_snippet = self.embedding(snippet)
        embedded_snippet = embedded_snippet.mean(dim=1)
        
        combined = torch.cat((embedded_title, embedded_snippet), dim=1)
        
        out = self.fc(combined)
        return out


In [150]:
class SnippetLightningModule(pl.LightningModule):
    def __init__(self, model, learning_rate, criterion):
        super(SnippetLightningModule, self).__init__()
        self.model = model
        self.criterion = criterion
        self.learning_rate = learning_rate

    def forward(self, title, snippet):
        return self.model(title, snippet)

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.learning_rate)
        return optimizer

    def training_step(self, batch, batch_idx):
        title, snippet, target = batch
        logits = self.model(title, snippet)
        loss = self.criterion(logits, target)
        self.log("train_loss", loss, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        title, snippet, target = batch
        logits = self.model(title, snippet)
        loss = self.criterion(logits, target)
        self.log("val_loss", loss, prog_bar=True)
        return loss


In [151]:
EMBEDDING_DIM = 300
EPOCHS = 5
LR = 1e-3

model_snippet = SnippetClassifier(VOCAB_SIZE, EMBEDDING_DIM, CLASSES_NUM)
criterion = torch.nn.CrossEntropyLoss()

snippet_module = SnippetLightningModule(model_snippet, LR, criterion)

trainer = pl.Trainer(
    logger=wandb_logger,
    accelerator="gpu" if torch.cuda.is_available() else "cpu",
    max_epochs=EPOCHS
)

trainer.fit(snippet_module, train_dataloader_snippet, val_dataloader_snippet)


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/home/user/myenv/lib/python3.11/site-packages/pytorch_lightning/loggers/wandb.py:396: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
/home/user/myenv/lib/python3.11/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:654: Checkpoint directory ./news-categorization/14y6jq8z/checkpoints exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]



  | Name      | Type              | Params | Mode 
--------------------------------------------------------
0 | model     | SnippetClassifier | 4.0 M  | train
1 | criterion | CrossEntropyLoss  | 0      | train
--------------------------------------------------------
4.0 M     Trainable params
0         Non-trainable params
4.0 M     Total params
15.806    Total estimated model params size (MB)
4         Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

...

Epoch 4: 100%|██████████| 2139/2139 [00:17<00:00, 122.92it/s, v_num=jq8z, train_loss=5.400, val_loss=10.50]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

Epoch 4: 100%|██████████| 2139/2139 [00:19<00:00, 108.05it/s, v_num=jq8z, train_loss=5.400, val_loss=10.40]

`Trainer.fit` stopped: `max_epochs=5` reached.


Epoch 4: 100%|██████████| 2139/2139 [00:20<00:00, 105.17it/s, v_num=jq8z, train_loss=5.400, val_loss=10.40]


## 1.5 Инференс и оценка качества моделей

Для каждой новости нам надо предсказывать несколько тэгов. То есть в нашем случае настоящее значение таргета это множество из тэгов $y_i = [tag1, tag2, tag3]$. Прогноз модели также множество из тэгов $\hat y_i = [tag1, tag4]$.

Будем считать метрики качества следующим образом (под $|A|$ имеется в виду мощность множества, то есть число элементов в нём):

$$
Precision = \frac{1}{n} \sum_{i = 1}^n \frac{|y_i \cap \hat{y}_i|}{|\hat{y}_i|}
$$

$$
Recall = \frac{1}{n} \sum_{i = 1}^n \frac{|y_i \cap \hat{y}_i|}{|y_i|}
$$

Также можно считать аналог Accuracy, но это не самая удачная идея, так как у нас в выборке огромное число нулей и эта метрика при любом разумном пороге для принятия решения будет очень высокой.

$$
Exact Match = \frac{1}{n} \cdot \frac{1}{k} \sum_{i = 1}^n \sum_{k=1}^K [y_{ij} = \hat{y}_{ij}]
$$

In [152]:
def precision(target, y_pred):
    num = ((y_pred == 1) & (target == 1)).sum(dim=1)
    denum = (y_pred == 1).sum(dim=1)
    return (num/(denum + 1e-5)).mean().item()

def recall(target, y_pred):
    num = ((y_pred == 1) & (target == 1)).sum(dim=1)
    denum = (target == 1).sum(dim=1)
    return (num/(denum + 1e-5)).mean().item()

def exact_match(target, y_pred):
    return (1.*(y_pred == target)).mean().item()

Построим прогноз на тестовой выборке.

In [153]:
test_dataloader = DataLoader(test_dataset, shuffle=False, batch_size=test_dataset.__len__())

for title, target in test_dataloader:
    logit = model_baseline(title)
    pred_prob = F.softmax(logit, dim=1)

assert pred_prob.shape[0] == test_dataset.__len__()

Теперь выбирая различное значение порога, мы можем получать разные предсказания. Если взять очень большое значение порога, то метрики сильно просядут, так как во многих документах никакого прогноза не будет построено вообще.

In [154]:
TRESHOLD = 0.01
y_pred = 1*(pred_prob > TRESHOLD)

print('Exact Match:', exact_match(target, y_pred))
print('Precision:', precision(target, y_pred))
print('Recall:', recall(target, y_pred))

Exact Match: 0.9954836964607239
Precision: 0.2893848717212677
Recall: 0.7641034126281738


In [155]:
TRESHOLD = 0.05
y_pred = 1*(pred_prob > TRESHOLD)
print('Exact Match:', exact_match(target, y_pred))
print('Precision:', precision(target, y_pred))
print('Recall:', recall(target, y_pred))

Exact Match: 0.9983219504356384
Precision: 0.5199970006942749
Recall: 0.6185795664787292


In [156]:
TRESHOLD = 0.9
y_pred = 1*(pred_prob > TRESHOLD)

print('Exact Match:', exact_match(target, y_pred))
print('Precision:', precision(target, y_pred))
print('Recall:', recall(target, y_pred))

Exact Match: 0.9983181357383728
Precision: 0.00599716417491436
Recall: 0.005377625115215778


Дальше мы будем строить довольно много прогнозов. Давайте напишем код для их строительства в виде функции. Обратите внимание, что на модели со снипетом она упадёт. Когда вы доберётесь до строительства прогнозов, функцию придётся немного модернизировать.

In [157]:
def get_predict(model, dataset):
    dataloader = DataLoader(dataset, shuffle=False, batch_size=len(dataset))
    
    for batch in dataloader:
        if len(batch) == 2:
            title, target = batch
            logit = model(title)
        elif len(batch) == 3:
            title, snippet, target = batch
            logit = model(title, snippet)
        
        pred_prob = F.softmax(logit, dim=1)
    
    assert pred_prob.shape[0] == len(dataset)
    return pred_prob, target


__[0.5 балла] Задание 5:__ Какая метрика для нас в этой задаче важнее? Точность или полнота? Почему?

__ваш ответ:__ в данной задаче важнее полнота, так как нам нужно каждая статья правильно классифицирована

- Напишите функцию, которая будет подбирать оптимальное значение порога, оптимизирующее выбранную вами метрику.
- Подберите значение порога на валидационной выборке.
- Сравните модель со сниппетами и без сниппетов, используя выбранную вами метрику при оптимальном значении порога на тестовой выборке.
- Какая из них оказалась лучше?

In [158]:
def find_optimal_threshold(y_true, y_probs, metric='f1', step=0.01):
    thresholds = np.arange(0.0, 1.0, step)
    best_threshold = 0.0
    best_metric = -1.0
    
    for threshold in thresholds:
        y_pred = (y_probs > threshold).float()
        if metric == 'recall':
            current_metric = recall(y_true, y_pred)
        elif metric == 'precision':
            current_metric = precision(y_true, y_pred)
        elif metric == 'f1':
            current_metric = f1_score(y_true, y_pred)
        
        if current_metric > best_metric:
            best_metric = current_metric
            best_threshold = threshold
            
    return best_threshold, best_metric

def f1_score(target, y_pred):
    precision_val = precision(target, y_pred)
    recall_val = recall(target, y_pred)
    if precision_val + recall_val == 0:
        return 0.0
    return 2 * (precision_val * recall_val) / (precision_val + recall_val)

In [159]:
val_probs_baseline, val_targets_baseline = get_predict(model_baseline, val_dataset)

val_probs_snippet, val_targets_snippet = get_predict(model_snippet, val_dataset_snippet)

optimal_threshold_baseline, best_f1_baseline = find_optimal_threshold(
    val_targets_baseline, 
    val_probs_baseline, 
    metric='f1', 
    step=0.01
)

print(f"Оптимальный порог для базовой модели: {optimal_threshold_baseline:.2f}, F1-Score: {best_f1_baseline:.4f}")

optimal_threshold_snippet, best_f1_snippet = find_optimal_threshold(
    val_targets_snippet, 
    val_probs_snippet, 
    metric='f1', 
    step=0.01
)

print(f"Оптимальный порог для модели со сниппетом: {optimal_threshold_snippet:.2f}, F1-Score: {best_f1_snippet:.4f}")


Оптимальный порог для базовой модели: 0.06, F1-Score: 0.6522
Оптимальный порог для модели со сниппетом: 0.06, F1-Score: 0.6660


In [160]:
def apply_threshold(y_probs, threshold):
    return (y_probs > threshold).float()

test_probs_baseline, test_targets_baseline = get_predict(model_baseline, test_dataset)
y_pred_baseline = apply_threshold(test_probs_baseline, optimal_threshold_baseline)
recall_baseline_test = recall(test_targets_baseline, y_pred_baseline)
print(f"Recall на тестовой выборке для базовой модели: {recall_baseline_test:.4f}")

test_probs_snippet, test_targets_snippet = get_predict(model_snippet, test_dataset_snippet)
y_pred_snippet = apply_threshold(test_probs_snippet, optimal_threshold_snippet)
recall_snippet_test = recall(test_targets_snippet, y_pred_snippet)
print(f"Recall на тестовой выборке для модели со сниппетом: {recall_snippet_test:.4f}")


Recall на тестовой выборке для базовой модели: 0.5959
Recall на тестовой выборке для модели со сниппетом: 0.6010


In [161]:
def apply_threshold(y_probs, threshold):
    return (y_probs > threshold).float()

test_probs_baseline, test_targets_baseline = get_predict(model_baseline, test_dataset)
y_pred_baseline = apply_threshold(test_probs_baseline, optimal_threshold_baseline)
recall_baseline_test = recall(test_targets_baseline, y_pred_baseline)
precision_baseline_test = precision(test_targets_baseline, y_pred_baseline)
f1_baseline_test = f1_score(test_targets_baseline, y_pred_baseline)

print(f"Базовая модель на тестовой выборке:")
print(f"F1-Score: {f1_baseline_test:.4f}")
print(f"Precision: {precision_baseline_test:.4f}")
print(f"Recall: {recall_baseline_test:.4f}")

test_probs_snippet, test_targets_snippet = get_predict(model_snippet, test_dataset_snippet)
y_pred_snippet = apply_threshold(test_probs_snippet, optimal_threshold_snippet)
recall_snippet_test = recall(test_targets_snippet, y_pred_snippet)
precision_snippet_test = precision(test_targets_snippet, y_pred_snippet)
f1_snippet_test = f1_score(test_targets_snippet, y_pred_snippet)

print(f"\nМодель со сниппетом на тестовой выборке:")
print(f"F1-Score: {f1_snippet_test:.4f}")
print(f"Precision: {precision_snippet_test:.4f}")
print(f"Recall: {recall_snippet_test:.4f}")


Базовая модель на тестовой выборке:
F1-Score: 0.5669
Precision: 0.5406
Recall: 0.5959

Модель со сниппетом на тестовой выборке:
F1-Score: 0.5771
Precision: 0.5551
Recall: 0.6010


Со снипитом лучше, и пресижн выше и рекол выше.

UPD: Я поперезапускал с не фиксированным сидом, и то со снипетом то без лучше, так что можно сказать примерно одинаково


__[0.5 балла] Задание 6:__  Постройте прогнозы для отложенной выборки, которая представляет из себя пересечение сайта РИА-новостей и ВКонтакте. Проседает ли на ней качество модели? Насколько сильно?

In [162]:
df_oob.head()

,href,title_clean,target_tags
0,/20181231/1548960744.html,митрополит рязанский раскритиковал иронию судьбы,[1115]
1,/20181231/1548958617.html,на украине позавидовали стене на границе с крымом,"[1359, 942]"
2,/20181231/1548957394.html,в госдуме предложили отказаться от газа в жилы...,"[1556, 1556, 1201, 194, 973]"
3,/20181231/1548954909.html,названы лучшие средства от похмелья,[1115]
4,/20181231/1548957120.html,новогоднее поздравление порошенко разозлило по...,"[1359, 942]"


In [163]:
df_oob = df_oob.merge(
    df_vk[['href', 'snippet_clean']],
    on='href',
    how='left'
)

df_oob['snippet_clean'] = df_oob['snippet_clean'].fillna("#UNKN").apply(normalise_text)

print(df_oob.head())


                        href  \
0  /20181231/1548960744.html   
1  /20181231/1548958617.html   
2  /20181231/1548957394.html   
3  /20181231/1548954909.html   
4  /20181231/1548957120.html   

                                         title_clean  \
0   митрополит рязанский раскритиковал иронию судьбы   
1  на украине позавидовали стене на границе с крымом   
2  в госдуме предложили отказаться от газа в жилы...   
3                названы лучшие средства от похмелья   
4  новогоднее поздравление порошенко разозлило по...   

                    target_tags snippet_clean  
0                        [1115]          unkn  
1                   [1359, 942]          unkn  
2  [1556, 1556, 1201, 194, 973]          unkn  
3                        [1115]          unkn  
4                   [1359, 942]          unkn  


In [164]:
class OOBDatasetBaseline(Dataset):
    def __init__(self, titles, targets, vocab, vocab_size, max_title_len, max_classes):
        self.vocab = {word: idx for word, idx in vocab.items() if idx < vocab_size}
        self.max_classes = max_classes
        self.y = self.target_ohe(targets)
        self.X_title = self.create_text(titles, max_title_len)
    
    def target_ohe(self, target):
        y = torch.zeros((len(target), self.max_classes))
        for i, t in enumerate(target):
            y[[i]*len(t), t] = 1.0
        return y
    
    def create_text(self, texts, max_len):
        result = []
        for sent in texts:
            sent_tokenize = [self.vocab.get(item, 1) for item in word_tokenize(sent)]
            if len(sent_tokenize) >= max_len:
                sent_tokenize = sent_tokenize[:max_len]
            else:
                sent_tokenize += [0] * (max_len - len(sent_tokenize))
            result.append(sent_tokenize)
        return torch.tensor(result, dtype=torch.int)
    
    def __len__(self):
        return len(self.X_title)
    
    def __getitem__(self, idx):
        return (self.X_title[idx, :], self.y[idx])

class OOBDatasetSnippet(Dataset):
    def __init__(self, titles, snippets, targets, vocab, vocab_size, max_title_len, max_snippet_len, max_classes):
        self.vocab = {word: idx for word, idx in vocab.items() if idx < vocab_size}
        self.max_classes = max_classes
        self.y = self.target_ohe(targets)
        self.X_title = self.create_text(titles, max_title_len)
        self.X_snippet = self.create_text(snippets, max_snippet_len)
    
    def target_ohe(self, target):
        y = torch.zeros((len(target), self.max_classes))
        for i, t in enumerate(target):
            y[[i]*len(t), t] = 1.0
        return y
    
    def create_text(self, texts, max_len):
        result = []
        for sent in texts:
            sent_tokenize = [self.vocab.get(item, 1) for item in word_tokenize(sent)]
            if len(sent_tokenize) >= max_len:
                sent_tokenize = sent_tokenize[:max_len]
            else:
                sent_tokenize += [0] * (max_len - len(sent_tokenize))
            result.append(sent_tokenize)
        return torch.tensor(result, dtype=torch.int)
    
    def __len__(self):
        return len(self.X_title)
    
    def __getitem__(self, idx):
        return (self.X_title[idx, :], self.X_snippet[idx, :], self.y[idx])

oob_dataset_baseline = OOBDatasetBaseline(
    titles=df_oob.title_clean.values,
    targets=df_oob.target_tags.values,
    vocab=vocabulary,
    vocab_size=VOCAB_SIZE,
    max_title_len=MAX_TITLE_LEN,
    max_classes=CLASSES_NUM
)

oob_dataset_snippet = OOBDatasetSnippet(
    titles=df_oob.title_clean.values,
    snippets=df_oob.snippet_clean.values,
    targets=df_oob.target_tags.values,
    vocab=vocabulary,
    vocab_size=VOCAB_SIZE,
    max_title_len=MAX_TITLE_LEN,
    max_snippet_len=MAX_SNIPPET_LEN,
    max_classes=CLASSES_NUM
)

oob_dataloader_baseline = DataLoader(oob_dataset_baseline, shuffle=False, batch_size=len(oob_dataset_baseline), num_workers=4)
oob_dataloader_snippet = DataLoader(oob_dataset_snippet, shuffle=False, batch_size=len(oob_dataset_snippet), num_workers=4)


In [165]:
oob_probs_baseline, oob_targets_baseline = get_predict(model_baseline, oob_dataset_baseline)

oob_probs_snippet, oob_targets_snippet = get_predict(model_snippet, oob_dataset_snippet)


In [166]:
def apply_threshold(y_probs, threshold):
    return (y_probs > threshold).float()

y_pred_baseline_oob = apply_threshold(oob_probs_baseline, optimal_threshold_baseline)
y_pred_snippet_oob = apply_threshold(oob_probs_snippet, optimal_threshold_snippet)

recall_baseline_oob = recall(oob_targets_baseline, y_pred_baseline_oob)
precision_baseline_oob = precision(oob_targets_baseline, y_pred_baseline_oob)
f1_baseline_oob = f1_score(oob_targets_baseline, y_pred_baseline_oob)

recall_snippet_oob = recall(oob_targets_snippet, y_pred_snippet_oob)
precision_snippet_oob = precision(oob_targets_snippet, y_pred_snippet_oob)
f1_snippet_oob = f1_score(oob_targets_snippet, y_pred_snippet_oob)

print(f"Базовая модель на отложенной выборке:")
print(f"F1-Score: {f1_baseline_oob:.4f}")
print(f"Precision: {precision_baseline_oob:.4f}")
print(f"Recall: {recall_baseline_oob:.4f}")

print(f"\nМодель со сниппетом на отложенной выборке:")
print(f"F1-Score: {f1_snippet_oob:.4f}")
print(f"Precision: {precision_snippet_oob:.4f}")
print(f"Recall: {recall_snippet_oob:.4f}")


Базовая модель на отложенной выборке:
F1-Score: 0.5316
Precision: 0.5014
Recall: 0.5656

Модель со сниппетом на отложенной выборке:
F1-Score: 0.2546
Precision: 0.2513
Recall: 0.2580


Лог эксперементов https://api.wandb.ai/links/dima-malinitskiy-hse-university/tvjbhhwo

Давайте модернизируем наши архитектуры настолько, насколько это возможно.

__[0.9 балла] Бонусное задание 1:__ Попробуйте собрать более большую архитектуру. Например, сразу после слоя эмбеддингов вы можете попробовать добавить свёрточные слои (`Conv1D` свёртки). Поиграйте с оптимизатором и тп.

Опишите результаты своих экспериментов ниже. Расскажите, что конкретно вы делали и удалось ли вам улучшить качество модели. Все траектории обучения залоггируйте на WandB.   

__Ваш лог экспериментов:__

-
-
-

In [59]:
# your code here
# (⊙_⊙)

__[0.9 балла] Бонусное задание 2:__ Скачайте с сайта [Rusvectores](https://rusvectores.org/ru/models/) любые новостные word2vec эмбединги. Возьмите из модели эмбеддинги для всех слов, которые встречаются вв вашем словаре и добавьте их в модель первым слоем. Заморозьте этот слой и не обновляйте в нём веса. Если у вас в словаре есть слово, но его нет среди предобученных эмбеддингов, замените его на токен `#UNKN`.

__Ваш лог экспериментов:__

-
-
-

In [60]:
# your code here
# =^･ｪ･^=

__[1.5 балла] Задание 7:__ Зафайнтьюньте трансформер для решения задачи с помощью библиотеки `hugging face`. Выбор предобученной модели кратко обоснуйте.

Я выбрал DeepPavlov/rubert-base-cased. На hugging face, по запросу russian news data, нашел эту модельку, и понял что под эту задачу она очень хорошо подходит

__Ваш лог экспериментов:__

-
-
-

In [ ]:
import pandas as pd
from datasets import Dataset

train_dataset_hf = Dataset.from_pandas(df_train[['title_clean', 'snippet_clean', 'target_tags']])
val_dataset_hf = Dataset.from_pandas(df_val[['title_clean', 'snippet_clean', 'target_tags']])
test_dataset_hf = Dataset.from_pandas(df_test[['title_clean', 'snippet_clean', 'target_tags']])


In [62]:
from transformers import AutoTokenizer

model_name = "DeepPavlov/rubert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize_function(examples):
    return tokenizer(
        examples['title_clean'],
        examples['snippet_clean'],
        padding='max_length',
        truncation=True,
        max_length=128
    )

train_dataset_hf = train_dataset_hf.map(tokenize_function, batched=True)
val_dataset_hf = val_dataset_hf.map(tokenize_function, batched=True)
test_dataset_hf = test_dataset_hf.map(tokenize_function, batched=True)


Map: 100%|██████████| 20176/20176 [00:02<00:00, 8290.31 examples/s]


In [63]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=CLASSES_NUM,
    problem_type="multi_label_classification"
)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at DeepPavlov/rubert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [66]:
import wandb
from wandb_key import API_KEY 
from transformers import (
    AutoModelForSequenceClassification, 
    AutoTokenizer, 
    TrainingArguments, 
    Trainer, 
    EarlyStoppingCallback
)
from sklearn.metrics import f1_score, precision_score, recall_score
from datasets import Dataset
import torch
import numpy as np

wandb.login(key=API_KEY)

train_dataset_hf = Dataset.from_pandas(df_train[['title_clean', 'snippet_clean', 'target_tags']])
val_dataset_hf = Dataset.from_pandas(df_val[['title_clean', 'snippet_clean', 'target_tags']])
test_dataset_hf = Dataset.from_pandas(df_test[['title_clean', 'snippet_clean', 'target_tags']])

model_name = "DeepPavlov/rubert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize_function(examples):
    return tokenizer(
        examples['title_clean'],
        examples['snippet_clean'],
        padding="max_length",
        truncation=True,
        max_length=128
    )

train_dataset_hf = train_dataset_hf.map(tokenize_function, batched=True)
val_dataset_hf = val_dataset_hf.map(tokenize_function, batched=True)
test_dataset_hf = test_dataset_hf.map(tokenize_function, batched=True)

CLASSES_NUM = len(idx2tag)
num_labels = CLASSES_NUM

def encode_labels(examples):
    labels = np.zeros((len(examples['target_tags']), num_labels))
    for i, tags in enumerate(examples['target_tags']):
        for tag in tags:
            if tag < num_labels:
                labels[i][tag] = 1
    examples["labels"] = labels.tolist()
    return examples

train_dataset_hf = train_dataset_hf.map(encode_labels, batched=True)
val_dataset_hf = val_dataset_hf.map(encode_labels, batched=True)
test_dataset_hf = test_dataset_hf.map(encode_labels, batched=True)

train_dataset_hf = train_dataset_hf.remove_columns(['title_clean', 'snippet_clean', 'target_tags'])
val_dataset_hf = val_dataset_hf.remove_columns(['title_clean', 'snippet_clean', 'target_tags'])
test_dataset_hf = test_dataset_hf.remove_columns(['title_clean', 'snippet_clean', 'target_tags'])

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    probs = torch.sigmoid(torch.tensor(logits))
    y_pred = (probs > 0.5).int().numpy()
    y_true = labels
    precision = precision_score(y_true, y_pred, average='micro', zero_division=0)
    recall = recall_score(y_true, y_pred, average='micro', zero_division=0)
    f1 = f1_score(y_true, y_pred, average='micro', zero_division=0)
    return {"precision": precision, "recall": recall, "f1": f1}

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    report_to="wandb",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    fp16=True
)

model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=num_labels,
    problem_type="multi_label_classification"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset_hf,
    eval_dataset=val_dataset_hf,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
)

trainer.train()

test_results = trainer.evaluate(test_dataset_hf)
print(test_results)


wandb: WARNING Calling wandb.login() after wandb.init() has no effect.
Map: 100%|██████████| 20176/20176 [00:00<00:00, 134597.72 examples/s]
/home/user/myenv/lib/python3.11/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at DeepPavlov/rubert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference

Epoch,Training Loss,Validation Loss,Precision,Recall,F1
1,0.000200,0.004998,0.652406,0.606965,0.628866
2,0.000100,0.005239,0.801242,0.641791,0.712707
3,0.000100,0.004070,0.771277,0.721393,0.745501


{'eval_loss': 0.002748664002865553, 'eval_precision': 0.7010309278350515, 'eval_recall': 0.8717948717948718, 'eval_f1': 0.7771428571428571, 'eval_runtime': 75.6758, 'eval_samples_per_second': 266.611, 'eval_steps_per_second': 4.176, 'epoch': 3.0}


Сравните все обученные модели между собой на тестовой выборке.

## Часть 2: предсказание категорий (0.5 балла)

**[0.5 балла] Задание 8:** Возьмите датасет `df_vk` и для всех новостей из него предскажите категории с помощью лучшей, получившейся у вас модели.

In [68]:
print(df_vk.columns)

Index(['id', 'href', 'datetime', 'title', 'likes', 'comments', 'snippet',
       'title_clean', 'snippet_clean'],
      dtype='object')


In [ ]:
from datasets import Dataset

df_vk['target_tags'] = [[] for _ in range(df_vk.shape[0])]

vk_dataset_hf = Dataset.from_pandas(df_vk[['title_clean', 'snippet_clean', 'target_tags']])


In [75]:
print(vk_dataset_hf.column_names)


['title_clean', 'snippet_clean', 'target_tags']


In [77]:

def tokenize_function_infer(examples):
    return tokenizer(
        examples['title_clean'],
        examples['snippet_clean'],
        padding="max_length",
        truncation=True,
        max_length=128
    )

vk_dataset_hf = vk_dataset_hf.map(tokenize_function_infer, batched=True)


Map: 100%|██████████| 19928/19928 [00:01<00:00, 12236.33 examples/s]


In [ ]:
predictions = trainer.predict(vk_dataset_hf)
logits = predictions.predictions
probs = torch.sigmoid(torch.tensor(logits))

threshold = 0.5

y_pred = (probs > threshold).int().numpy()


На всякий случай сохраните табличку с получившимися у вас предсказаниями. Мало ли, вы не доделаете последнее задание, а потом захотите вернуться к нему. Не прогонять же обучение нейросети и инференс по второму кругу...

In [ ]:
def decode_predictions(y_pred, idx2tag):
    predictions = []
    for sample in y_pred:
        tags = [idx2tag[idx] for idx, val in enumerate(sample) if val == 1]
        predictions.append(tags)
    return predictions

predicted_tags = decode_predictions(y_pred, idx2tag)

df_vk['target_tags'] = predicted_tags


In [ ]:
df_vk.to_csv('vk_news_with_predictions.csv', index=False)
df_vk.to_parquet('vk_news_with_predictions.parquet', index=False)


## Продолжение дальше, в некст файле